In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [3]:
model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [4]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('Я люблю гулять со своей собакой', return_tensors='pt')

In [32]:
# import json
# dictionary = {i: tokenizer.decode(i) for i in range(50256)}

# with open('dict.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(dictionary, ensure_ascii=False, indent=4,))

In [83]:
def predict_tokens(s, n=5, l=1, top_k=0, top_p=1., temperature=1.):
    # encode context the generation is conditioned on
    input_ids = tokenizer.encode(s, return_tensors='pt')
    # activate beam search and early_stopping
    greedy_output = model.generate(input_ids, max_length=input_ids.shape[1]+l).to('cuda')
    beam_outputs = model.generate(
        input_ids,  
        max_length=input_ids.shape[1]+l, 
#         num_beams=n, 
        early_stopping=True,
        do_sample=True,
        num_return_sequences=n-1,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature
    ).to('cuda')

    return [tokenizer.decode(beam_output, skip_special_tokens=True)[len(s):].rstrip() for beam_output in torch.cat([greedy_output, beam_outputs], 0)]

In [84]:
predict_tokens('Я люблю')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[' тебя', ' девочек', ' свою', ' тебя', ' Тебя']

In [7]:
predict_tokens('Я хочу')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я хочу:
----------------------------------------------------------------------------------------------------
0: ','
1: ' сказать'
2: ' быть'
3: ' знать'
4: ' жить'


In [8]:
predict_tokens('Он')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Он:
----------------------------------------------------------------------------------------------------
0: ' был'
1: ' не'
2: ' сказал'
3: ','
4: ' уже'


In [9]:
predict_tokens('Hello')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Hello:
----------------------------------------------------------------------------------------------------
0: ','
1: '!'
2: ' World'
3: '\n'
4: ' world'


In [12]:
predict_tokens('Я хочу', l=50, top_k=10, top_p=0.95) 

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я хочу:
----------------------------------------------------------------------------------------------------
0: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива'
1: ', чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива'
2: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я…'
3: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я хочу'
4: ', чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я…'


In [29]:
predict_tokens('Вечерело. Проходя мимо старого засаленого кабака я вдруг', l=50, top_k=50, top_p=0.95, n=5)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Вечерело. Проходя мимо старого засаленого кабака я вдруг:
----------------------------------------------------------------------------------------------------
0:  поймал себя на мысли, что я не хочу выпить в этом захолустье, и это желание почему-то мне понравилось.

Проходя мимо какого-то грязного здания я остановился и подумал, что неплохо было бы помыться, как вдруг
1:  услышал скрип калитки, и увидел на ней силуэт. Это была женщина. Я быстро завернул за угол и увидел ее. Женщина была босая и в одной рубашке. Я увидел, как она сняла с головы свою соломенную шляпу, отбросила ее
2:  вспомнил:
- Ребята, а у Вас ничего не найдется выпить?
В ответ мне был гробовой взгляд.
Я тогда понял, что мне и здесь не рады.
В этот момент в кабаке появились люди в чёрных плащах.
3:  заметил, что в окне зажигается свет и кто-то выходит на улицу. Вскочив на свой велосипед я помчался на свет и через пару минут увидел того самого бродягу, который вышел за мной. Я его окликнул, но тот не обратил на меня
4: 

In [35]:
predict_tokens('Вечерело. Проходя мимо старого засаленого кабака я вдруг', l=50, top_k=50, top_p=0.90, n=5) 

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Вечерело. Проходя мимо старого засаленого кабака я вдруг:
----------------------------------------------------------------------------------------------------
0:  подумал: «А может это тот самый? Надо бы наведаться. Только сегодня как-то не хочется, а вдруг это тот самый? Я ведь так ничего о нем не знаю. Хотя, чего это я все о нем? Да и к чему
1:  заметил двух проституток, они курили одну сигарету. Одна из них была явно с улицы и в платье, как у шлюхи. Я не мог не подойти, познакомиться, но она сделала мне знак молчать. И я стоял молча, разглядывая ее платье
2:  увидел, что на дверях стоит объявление, извещающее о закрытии, да еще и с указанием времени.
- Блин, ну неееет… - протянул я и полез в карман за мелочью.
- Деньги есть?
Я с
3:  остановился, и не просто остановился, а замер.

Там, за стойкой, кто-то сидел и пил. Я заглянул за стойку и замер от ужаса:

— Сюда нельзя, не смотри, — сказала Оля.
4:  остановился, будто налетел на стену, и долго пытался вспомнить, где же я видел его. 

In [85]:
string = ''
variants = []
n = 5

def str2int(s, lim):
    i = -1
    try:
        i = int(s)
    except:
        pass
    return i

while True:
    print(string)
    print('\n'.join([f'{inx}:{var}' for inx, var in enumerate(variants)]))
    msg = input()
    
    if msg == 'stop':
        print(string)
        break
        
    i = str2int(msg, n)
    if i != -1:
        string += variants[i] if variants else ''
    else:
        string += msg
        
    variants = predict_tokens(string)
    

 Я иду по


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я иду по
0: улице
1: парку
2: ки
3: городу
4: опасной


 1


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

 0


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я иду по парку, вспоминаю эпизоды из детства.Эти воспоминания довольно необычные, но если задуматься, то этодаже очень смешные.
0:
1:Ре
2: Вот
3: Он
4: Мы


 stop


Я иду по парку, вспоминаю эпизоды из детства.Эти воспоминания довольно необычные, но если задуматься, то этодаже очень смешные.


In [ ]:
0